In [1]:
import torch

torch.cuda.empty_cache()
print(f"GPU: {torch.cuda.get_device_name(0)}")

from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    TextStreamer,
)
import transformers

from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM


GPU: NVIDIA GeForce RTX 4090


2024-01-08 15:41:16.580929: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-08 15:41:16.599822: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-08 15:41:16.599843: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-08 15:41:16.599858: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-08 15:41:16.604075: I tensorflow/core/platform/cpu_feature_g

In [2]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)
inf_model_path="zephyr-7b-padding-full"
inf_model=AutoModelForCausalLM.from_pretrained(inf_model_path, device_map="auto")
inf_tokenizer = AutoTokenizer.from_pretrained(inf_model_path)
streamer = TextStreamer(inf_tokenizer)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
dataset = load_dataset("gsm8k",'main')
print(dataset['test'][0])

{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?", 'answer': 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'}


In [5]:
# Run text generation pipeline with our next model
import random
# 将数据集转换为 Python 列表
data_list = list(dataset['test'])  # 假设您想要从训练集中抽取数据
# 从列表中随机选择一条数据
random_data_point = random.choice(data_list)
print(f"question: {random_data_point['question']}\n")
print(f"answer: {random_data_point['answer']}\n")
# 现在 random_data_point 包含了您随机选择的一条数据
# print(random_data_point['question'])
print("----model_inferencing----")

prompt = "Dean's mother gave him $28 to go to the toy store. Dean bought 6 toy cars and 5 teddy bears. Each toy car cost $2 and each teddy bear cost $1. His mother then feels generous and decides to give him an extra $10. How much money does Dean have left?"
pipe = pipeline(task="text-generation", model=inf_model, tokenizer=inf_tokenizer, max_length=300, streamer=streamer)
result = pipe(f"<|user|>{prompt}</s>\n")
print(result[0]['generated_text'])

question: Mandy owes Benedict $100. They agreed to have monthly interest of 2%. If Mandy was able to pay it after 3 months, how much should she give to Benedict?

answer: The monthly interest is $100 x 2/100 = $<<100*2/100=2>>2.
After 3 months, the total interest amount to $2 x 3 = $<<2*3=6>>6.
Therefore, Mandy should give Benedict a total of $100 + $6 = $<<100+6=106>>106.
#### 106

----model_inferencing----
<|user|>Dean's mother gave him $28 to go to the toy store. Dean bought 6 toy cars and 5 teddy bears. Each toy car cost $2 and each teddy bear cost $1. His mother then feels generous and decides to give him an extra $10. How much money does Dean have left?</s> 
<|assistant|>
Dean bought 6 toy cars at $2 each for a total of $6.
He bought 5 teddy bears at $1 each for a total of $5.
The total cost of the toys is $6 + $5 = $<<6+5=11>>11.
Dean has $28 - $11 = $<<28-11=17>>17 left.
His mother then gives him $10, so he has $17 + $10 = $<<17+10=27>>27.
#### 27 dollars
#### 27.00 dollars
###